#### Module imports

In [ ]:
import torch
import numpy as np
from numpy import asarray
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import permute
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import pickle
import pandas as pd
from tqdm import tqdm

import sys
sys.path.insert(0,'../')
import label_tools as lt

#### Loading Files

load pre processed files

In [ ]:
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_pred_polygon_224/')
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_pred_square_205/')
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_gt_polygon_296/')
#imgs_path = Path(''/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_gt_square_296/'')

imgs_path = Path('/home/richard/data/Schiefer/combine/preprocessed_512_clipped_pred_polygon_1126/')
assert imgs_path.is_dir()
files = sorted(imgs_path.glob('*.png'))

randomizer = np.random.RandomState(seed=99833)
randomizer.shuffle(files)

assert len(files) == int(str(imgs_path).split('_')[5]) # all files are found
print("First 10 files are: {}".format([x.name for x in files[:10]]))

#### Loading Images

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def alter_image(img_name):
    image = Image.open(img_name)
    image_tensor = normalize(to_tensor(image)).unsqueeze(0)
    image_tensor = image_tensor.reshape(1,3,512,512)
    return image_tensor

def load_images_as_tensors(paths):
    images = [alter_image(image) for image in paths]
    return images

image_tensors = load_images_as_tensors(files)
assert len(image_tensors) == int(str(imgs_path).split('_')[5])

#### Extracting labels from filenames

In [ ]:
def extract_labels(files): return [filename.stem.split('_')[4] for filename in files]
labels = extract_labels(files)
print('first 10 labels: {}'.format(labels[:10]))

In [ ]:
print(set(labels))

In [ ]:
# change labels from number to species (of collab)
update = {
    '4':'Fagus_sylvatica',
    '5':'Fraxinus_excelsior',
    '6':'Quercus_spec',
    '8':'deadwood',
    '10':'Abies_alba',
    '11':'Larix_decidua',
    '12':'Picea_abies',
    '13':'Pinus_sylvestris',
    '14':'Pseudotsuga_menziesii'
}

updated_labels = (pd.Series(labels)).map(update)
species_labels = list(updated_labels)
labels = species_labels
print('first 10 labels: {}'.format(labels[:10]))

In [ ]:
# # change labels from number to species
# update = {
#     '4':'Fagus_sylvatica',
#     '8':'deadwood',
#     '10':'Abies_alba',
#     '12':'Picea_abies'
# }

# updated_labels = (pd.Series(labels)).map(update)
# species_labels = list(updated_labels)
# labels = species_labels
# print('first 10 labels: {}'.format(labels[:10]))

#### Label encoding

Standardize encodings of labels to make analysis easier afterwards.

In [ ]:
le = lt.CustomLabelEncoder()
le.fit(labels, sorter=lambda x: x.upper())

labels_int = le.transform(labels[:10])
labels_str = le.inverse_transform(labels_int)

label_dir = Path('/home/richard/data/Schiefer/feature_extraction/labels')
filename = Path('ResNet152_polygon_pred_label_encodings_512_' + str(imgs_path).split('_')[5] + '.pickle')
with open(label_dir / filename, 'wb') as f:
    pickle.dump(le, f)

print('label encodings: {}'.format(le.mapper))
print('first 10 integer labels: {}'.format(labels_int))
print('first 10 string labels: {}'.format(labels_str))

In [ ]:
print(le.transform(labels))

#### Feature Extraction

Load the ResNet152 model

In [ ]:
# Load the pretrained model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)# Use the model object to select the desired layer
layer = model._modules.get('avgpool')
# Set model to evaluation mode
model.eval()

In [ ]:
def get_feature_vector(image_tensor):
    # a dict to store the activations
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    # register forward hooks on the layers of choice
    h1 = model.avgpool.register_forward_hook(getActivation('avgpool'))

    # forward pass -- getting the outputs
    out = model(image_tensor)

    # detach the hooks
    h1.remove()
    
    feature = torch.squeeze(activation['avgpool'])
    feature = torch.unsqueeze(feature,dim=0)
    return feature

# check
feature = get_feature_vector(image_tensors[0])
print(feature.shape)

Get all features of the folder

In [ ]:
from tqdm.notebook import tqdm

def concat_tensors(features):
    fc = torch.cat(features)
    fc = fc.cpu().detach().numpy()
    return fc

def get_features(image_tensors):
    features = [get_feature_vector(tensor) for tensor in tqdm(image_tensors)]
    features = concat_tensors(features)
    return features

fc = get_features(image_tensors)
print(fc.shape)

Save features

In [ ]:
results = {'filename': files,
           'features': fc,
           'labels': labels,
           'layer_name': 'fc'}

feature_dir = Path('/home/richard/data/Schiefer/feature_extraction/features')
feature_filename = Path('ResNet152_polygon_pred_512_' + str(imgs_path).split('_')[5] + '.pickle')
print(feature_dir / feature_filename)
Path(feature_dir).mkdir(parents=True, exist_ok=True)
with open(feature_dir / feature_filename, 'wb') as f:
    pickle.dump(results, f)

In [ ]:
# test
from pathlib import Path
from glob import glob
images_path = '/home/richard/data/Schiefer/pred_polygon_clipped_raster_files/'
out_dir = Path(images_path).parent / Path('preprocessed_' + 'None' + '_clahe-denoising_' + 'clipped_pred_' + 'polygon' + '_' + str(204))
print(out_dir)

print(str(out_dir).split('_')[2])
import os
os.path.isdir(out_dir)

In [ ]:
from pathlib import Path
en_path = '/home/richard/data/Schiefer/label_encodings/effnet_clahe-denoising_polygon_pred.pickle'
ident = str(Path(en_path).stem).split('_')[0] + '_' + str(en_path).split('_')[2]
print(ident)
print(Path(en_path).stem)